In [53]:
import json
import requests
import shutil
import time
from datetime import datetime
from tqdm import tqdm_notebook as tqdm

In [89]:
start_time = str(1405982173)
end_time = str(1595370973)
url = 'http://api.pushshift.io/reddit/submission/search/?sort=desc&subreddit=RoastMe&limit=1000&after={}&before='

In [90]:
r = requests.get(url)
data = json.loads(r.text)
data['data'][:5]

[{'all_awardings': [],
  'allow_live_comments': False,
  'author': 'AnshajSingh',
  'author_flair_background_color': '',
  'author_flair_css_class': 'roastlink',
  'author_flair_richtext': [],
  'author_flair_text': 'http://redd.it/hv2l2y',
  'author_flair_text_color': 'dark',
  'author_flair_type': 'text',
  'author_fullname': 't2_173cd9cb',
  'author_patreon_flair': False,
  'author_premium': False,
  'awarders': [],
  'can_mod_post': False,
  'contest_mode': False,
  'created_utc': 1595313856,
  'domain': 'i.imgur.com',
  'full_link': 'https://www.reddit.com/r/RoastMe/comments/hv2l2y/your_comments_dont_even_feel_bad_anymoreprove_me/',
  'gildings': {},
  'id': 'hv2l2y',
  'is_crosspostable': True,
  'is_meta': False,
  'is_original_content': False,
  'is_reddit_media_domain': False,
  'is_robot_indexable': True,
  'is_self': False,
  'is_video': False,
  'link_flair_background_color': '',
  'link_flair_richtext': [],
  'link_flair_text_color': 'dark',
  'link_flair_type': 'text',
  

In [91]:
data['data'][0]['author']

'AnshajSingh'

In [108]:
start=1388615773
end=1595370973
previous_epoch = start
all_posts = {}
c = 0
for epoch in tqdm(range(start, end, 60*60*24*2)): #get results every 2 days
    if c%100==0:
        print(datetime.fromtimestamp(epoch))
    r = requests.get(url.format(previous_epoch) + str(epoch))
    data = json.loads(r.text)
    for entry in data['data']:
        if entry['score'] > 20 and entry['num_comments'] > 2 and entry['id'] not in all_posts:
            all_posts[entry['id']] = entry['url']
    time.sleep(1)
    previous_epoch = epoch
    c+=1

<ipython-input-108-a3802da6abf1>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm(range(start, end, 60*60*24*2)): #get results every 2 days


2014-01-01 17:36:13
2014-07-20 18:36:13
2015-02-05 17:36:13
2015-08-24 18:36:13
2016-03-11 17:36:13
2016-09-27 18:36:13
2017-04-15 18:36:13
2017-11-01 18:36:13
2018-05-20 18:36:13
2018-12-06 17:36:13
2019-06-24 18:36:13
2020-01-10 17:36:13



In [109]:
len(list(all_posts.keys()))

4761

In [110]:
key = list(all_posts.keys())[5]
(key, all_posts[key])

('3c082j', 'http://imgur.com/1qAyCSA')

In [126]:
def get_root_comments(_id, score_threshold=10, top_n=5):
    comments = []
    url = 'https://api.pushshift.io/reddit/comment/search/?sort_type=score&sort=desc&link_id={}&subreddit=RoastMe&limit=1000'.format(_id)
    r = requests.get(url)
    data = json.loads(r.text)
    for entry in data['data']:
        if entry['parent_id'][:3] == 't3_': # making sure it's a root comment
            if entry['score'] > score_threshold:
                comments.append([entry['body'],entry['score']])
    if len(comments)>5:
        return comments[:5]
    return comments
get_root_comments('h7kzpw')

[]

In [113]:
def download_img(filename, url):
    response = requests.get(url, stream=True)
    with open('./faces/{}.png'.format(filename), 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response

In [115]:
for c, key in tqdm(enumerate(list(all_posts.keys()))):
    download_img(key, all_posts[key])

<ipython-input-115-1bd1607c41b4>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for c, key in tqdm(enumerate(list(all_posts.keys()))):


In [86]:
'''
Sources:
http://opencv.willowgarage.com/documentation/python/cookbook.html
http://www.lucaamore.com/?p=638
'''

#Python 2.7.2
#Opencv 2.4.2
#PIL 1.1.7

import cv2 #Opencv
import PIL.Image #Image from PIL
import glob
import os

def DetectFace(image, faceCascade, returnImage=False):
    # This function takes a grey scale cv image and finds
    # the patterns defined in the haarcascade function
    # modified from: http://www.lucaamore.com/?p=638

    #variables    
    min_size = (20,20)
    haar_scale = 1.1
    min_neighbors = 3
    haar_flags = 0

    # Equalize the histogram
    cv2.EqualizeHist(image, image)

    # Detect the faces
    faces = cv2.HaarDetectObjects(
            image, faceCascade, cv2.CreateMemStorage(0),
            haar_scale, min_neighbors, haar_flags, min_size
        )

    # If faces are found
    if faces and returnImage:
        for ((x, y, w, h), n) in faces:
            # Convert bounding box to two CvPoints
            pt1 = (int(x), int(y))
            pt2 = (int(x + w), int(y + h))
            cv2.Rectangle(image, pt1, pt2, cv2.RGB(255, 0, 0), 5, 8, 0)

    if returnImage:
        return image
    else:
        return faces

def pil2cvGrey(pil_im):
    # Convert a PIL image to a greyscale cv image
    # from: http://pythonpath.wordpress.com/2012/05/08/pil-to-opencv-image/
    pil_im = pil_im.convert('L')
    cv_im = cv.CreateImageHeader(pil_im.size, cv.IPL_DEPTH_8U, 1)
    cv2.SetData(cv_im, pil_im.tostring(), pil_im.size[0]  )
    return cv_im

def cv2pil(cv_im):
    # Convert the cv image to a PIL image
    return Image.fromstring("L", cv2.GetSize(cv_im), cv_im.tostring())

def imgCrop(image, cropBox, boxScale=1):
    # Crop a PIL image with the provided box [x(left), y(upper), w(width), h(height)]

    # Calculate scale factors
    xDelta=max(cropBox[2]*(boxScale-1),0)
    yDelta=max(cropBox[3]*(boxScale-1),0)

    # Convert cv box to PIL box [left, upper, right, lower]
    PIL_box=[cropBox[0]-xDelta, cropBox[1]-yDelta, cropBox[0]+cropBox[2]+xDelta, cropBox[1]+cropBox[3]+yDelta]

    return image.crop(PIL_box)

def faceCrop(imagePattern,boxScale=1):
    # Select one of the haarcascade files:
    #   haarcascade_frontalface_alt.xml  <-- Best one?
    #   haarcascade_frontalface_alt2.xml
    #   haarcascade_frontalface_alt_tree.xml
    #   haarcascade_frontalface_default.xml
    #   haarcascade_profileface.xml
    faceCascade = cv2.Load('haarcascade_frontalface_alt.xml')

    imgList=glob.glob(imagePattern)
    if len(imgList)<=0:
        print('No Images Found')
        return

    for img in imgList:
        pil_im=Image.open(img)
        cv_im=pil2cvGrey(pil_im)
        faces=DetectFace(cv_im,faceCascade)
        if faces:
            n=1
            for face in faces:
                croppedImage=imgCrop(pil_im, face[0],boxScale=boxScale)
                fname,ext=os.path.splitext(img)
                croppedImage.save(fname+'_crop'+str(n)+ext)
                n+=1
        else:
            print('No faces found:', img)

def test(imageFilePath):
    pil_im=Image.open(imageFilePath)
    cv_im=pil2cvGrey(pil_im)
    # Select one of the haarcascade files:
    #   haarcascade_frontalface_alt.xml  <-- Best one?
    #   haarcascade_frontalface_alt2.xml
    #   haarcascade_frontalface_alt_tree.xml
    #   haarcascade_frontalface_default.xml
    #   haarcascade_profileface.xml
    faceCascade = cv2.Load('haarcascade_frontalface_alt.xml')
    face_im=DetectFace(cv_im,faceCascade, returnImage=True)
    img=cv2pil(face_im)
    img.show()
    img.save('test.png')


# Test the algorithm on an image
#test('testPics/faces.jpg')

# Crop all jpegs in a folder. Note: the code uses glob which follows unix shell rules.
# Use the boxScale to scale the cropping area. 1=opencv box, 2=2x the width and height
faceCrop('faces/*.png',boxScale=1)

AttributeError: module 'cv2.cv2' has no attribute 'Load'